In [1]:
import os
import polars as pl
from bs4 import BeautifulSoup

In [19]:
odkud = 'downloads/fdb'
stazene = os.listdir(odkud)

In [11]:
def vysosni_program(soubor):
    
    with open(os.path.join(odkud,soubor), 'r', encoding='utf-8') as zdroj:
        soup = BeautifulSoup(zdroj.read())
        
        kanaly = dict(enumerate([x.text.strip() for x in soup.find(id='st_razeni').find_all('li')]))
        programy = soup.find(class_='tbl_program_block')
        
        denni_program = []
        for radek in programy.find_all('tr'):
            for cislo, sloupec in dict(enumerate(radek.find_all('td'))).items():
                for zaznam in sloupec.find_all(class_='next_time'):
                    porad = {}
                    porad['kanal'] = kanaly[cislo]
                    porad['datum'] = soup.find('input', id='datepicker').get('value')
                    porad['cas'] = zaznam.find(class_='time_info').contents[0].text
                    try:
                        porad['id'] = zaznam.find('a', href=True)['href']
                    except:
                        pass
                    try:
                        if zaznam.find(class_='text_info').contents[0].text == ' ':
                            porad['nazev'] = zaznam.find(class_='text_info').contents[1].text.strip()
                        else:
                            porad['nazev'] = zaznam.find(class_='text_info').contents[0].text.strip()
                    except:
                        pass
                    try:
                        porad['extra'] = zaznam.find('span', class_=lambda value: value is None).text.strip()
                    except:
                        pass
                    try:
                        porad['noticka'] = zaznam.find(class_='txt_hint').text.strip()
                    except:
                        pass
                    denni_program.append(porad)

    return denni_program

In [13]:
test = vysosni_program('fdb_2017-04-15_1-ct1-ct2-nova-prima-nova-action-prima-hd-ct-3.html')
print(len(test))
print(test)

140
[{'kanal': 'ČT1', 'datum': '15.04.2017', 'cas': '05:15', 'nazev': 'Na forbíně TM', 'noticka': 'Tak trochu jiná talk show Tomáše Matonohy s jeho hosty L. Sobotou a B. Kleplem na prknech brněnského divadla Reduta.'}, {'kanal': 'ČT1', 'datum': '15.04.2017', 'cas': '06:00', 'id': './../film/kouzelne-bylinky/121723', 'nazev': 'Kouzelné bylinky', 'extra': '17.03.2025 v 08:25 (Spektrum Home)18.03.2025 v 03:55 (Spektrum Home)19.03.2025 v 03:55 (Spektrum Home)20.03.2025 v 03:55 (Spektrum Home)21.03.2025 v 03:55 (Spektrum Home)22.03.2025 v 03:55 (Spektrum Home)24.03.2025 v 08:25 (Spektrum Home)25.03.2025 v 03:55 (Spektrum Home)', 'noticka': 'Další díl/opakování:17.03.2025 v 08:25 (Spektrum Home)18.03.2025 v 03:55 (Spektrum Home)19.03.2025 v 03:55 (Spektrum Home)20.03.2025 v 03:55 (Spektrum Home)21.03.2025 v 03:55 (Spektrum Home)22.03.2025 v 03:55 (Spektrum Home)24.03.2025 v 08:25 (Spektrum Home)25.03.2025 v 03:55 (Spektrum Home)'}, {'kanal': 'ČT1', 'datum': '15.04.2017', 'cas': '06:25', 'id'

In [4]:
kam = "data_raw/fdb"
if not os.path.exists(kam):
    os.makedirs(kam)

In [23]:
oscrapovane = set(os.listdir(kam))
pocitadlo = len(oscrapovane)

In [6]:
def uloz_program(program, soubor):
    with open(os.path.join(kam, soubor.replace('.html','.json')), 'w+', encoding='utf-8') as vystup:
        if isinstance(program, list):
            for item in program:
                vystup.write(json.dumps(item) + '\n')
        else:
            vystup.write(json.dumps(program) + '\n')

In [7]:
import datetime

In [25]:
for x in stazene:
    if x.replace('.html','.json') not in oscrapovane:
        pocitadlo += 1
        if pocitadlo % 50 == 0:
            print(f"{datetime.datetime.now().strftime("%H:%M:%S")} {pocitadlo}/{len(stazene)} {x}")
        try:
            vysosnuto = vysosni_program(x)
            uloz_program(vysosnuto, x)
        except Exception as e:
            print(e)

'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
[Errno 13] Permission denied: 'downloads/fdb\\ne'
